# 🚀 Proyecto Integrador 2: Pipeline Near Real-Time, Scheduling y Alertas

Este proyecto consolida el nivel Junior con un pipeline que monitorea una carpeta de entrada, valida y transforma nuevos archivos CSV en tiempo casi real, carga resultados, genera métricas, registra logs, implementa reintentos con backoff y envía notificaciones (stub).

Objetivos:
- ✅ Monitoreo de archivos nuevos (file watcher)
- ✅ Validación de esquema y calidad
- ✅ Transformación y enriquecimiento
- ✅ Carga a SQLite y Parquet
- ✅ Métricas y logging estructurado
- ✅ Reintentos con backoff e idempotencia
- ✅ Notificaciones (Slack/Email stub)
- ✅ Scheduling simple sin dependencias externas

## 1. Configuración y Estructura

In [ ]:
import os, json, time, sqlite3, logging, traceback, hashlib
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import pandas as pd
import numpy as np

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
INPUT_DIR = os.path.join(BASE_DIR, 'ingest', 'incoming')
CHKPT_DIR = os.path.join(BASE_DIR, 'ingest', 'checkpoints')
OUT_DIR = os.path.join(BASE_DIR, 'outputs', 'proyecto_2')
LOGS_DIR = os.path.join(BASE_DIR, 'logs')

for d in [INPUT_DIR, CHKPT_DIR, OUT_DIR, LOGS_DIR]:
    os.makedirs(d, exist_ok=True)

DB_PATH = os.path.join(OUT_DIR, 'near_rt_analytics.db')
PARQUET_DIR = os.path.join(OUT_DIR, 'parquet')
os.makedirs(PARQUET_DIR, exist_ok=True)

# Logging
logging.basicConfig(level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(LOGS_DIR, 'proyecto_integrador_2.log'), encoding='utf-8'),
        logging.StreamHandler()
    ])
logger = logging.getLogger('near_rt_pipeline')
logger.info('✅ Entorno inicializado')
print('BASE_DIR:', BASE_DIR)
print('INPUT_DIR:', INPUT_DIR)
print('OUT_DIR:', OUT_DIR)

### 🏗️ Arquitectura de Pipeline Near Real-Time

**Concepto:** Pipeline que procesa archivos automáticamente al detectar nuevos en carpeta monitoreada.

**Estructura de directorios:**
- **ingest/incoming/**: Archivos nuevos llegan aquí (trigger)
- **ingest/checkpoints/**: Metadata de archivos procesados (evita duplicados)
- **outputs/proyecto_2/**: Datos transformados (DB + Parquet)
- **logs/**: Trazabilidad de ejecución

**Patrón:**
1. Watchdog monitorea incoming/
2. Detecta nuevo archivo → valida → transforma → carga
3. Guarda checkpoint para idempotencia

**Uso:** Pipelines batch-streaming, ingesta continua sin colas MQ complejas.

## 2. Utilidades: Hash, Checkpoints, Notificaciones (stub)

In [ ]:
def file_md5(path: str) -> str:
    h = hashlib.md5()
    with open(path, 'rb') as f:
        for chunk in iter(lambda: f.read(8192), b''):
            h.update(chunk)
    return h.hexdigest()

def chkpt_path(filename: str) -> str:
    base = os.path.basename(filename)
    return os.path.join(CHKPT_DIR, base + '.json')

def is_processed(filename: str, md5sum: str) -> bool:
    p = chkpt_path(filename)
    if not os.path.exists(p):
        return False
    try:
        data = json.load(open(p, 'r', encoding='utf-8'))
        return data.get('md5') == md5sum
    except Exception:
        return False

def save_chkpt(filename: str, md5sum: str, rows: int, status: str, error: Optional[str]=None):
    p = chkpt_path(filename)
    payload = {
        'filename': os.path.basename(filename),
        'md5': md5sum,
        'rows': rows,
        'status': status,
        'error': error,
        'timestamp': datetime.utcnow().isoformat()
    }
    json.dump(payload, open(p, 'w', encoding='utf-8'), ensure_ascii=False, indent=2)

def notify(message: str, level: str = 'info') -> None:
    """
    Stub de notificación: aquí iría Slack webhook o email SMTP.
    Para Slack: POST a https://hooks.slack.com/services/... con {'text': message}
    Para Email: usar smtplib con credenciales seguras (no hardcodear).
    """
    levels = {'info': logging.INFO, 'warning': logging.WARNING, 'error': logging.ERROR}
    logger.log(levels.get(level, logging.INFO), f'🔔 {message}')

print('✅ Utilidades listas')

### 🔒 Idempotencia: Checkpoints y Hash MD5

**Concepto:** Evitar procesar el mismo archivo múltiples veces mediante checksums y metadata.

**Componentes:**
- **MD5 Hash:** Firma única del contenido del archivo
- **Checkpoint JSON:** Almacena {filename, md5, timestamp, status}
- **is_processed():** Verifica si archivo ya fue procesado

**Flujo:**
1. Calcular MD5 del archivo nuevo
2. Buscar checkpoint existente
3. Si MD5 coincide → skip (ya procesado)
4. Si no → procesar y guardar checkpoint

**Importancia:** Pipeline idempotente permite reintentos sin duplicar datos (requisito producción).

## 3. Validación de Esquema y Transformación

In [ ]:
EXPECTED_COLUMNS = {
    'venta_id': 'int64',
    'cliente_id': 'int64',
    'producto_id': 'int64',
    'cantidad': 'int64',
    'precio_unitario': 'float64',
    'total': 'float64',
    'fecha_venta': 'datetime64[ns]',
    'metodo_pago': 'object',
    'estado': 'object'
}

def read_csv_safe(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    # Normalización mínima
    if 'fecha_venta' in df.columns:
        df['fecha_venta'] = pd.to_datetime(df['fecha_venta'], errors='coerce')
    return df

def validate_schema(df: pd.DataFrame) -> Tuple[bool, List[str]]:
    errors = []
    for col, dtype in EXPECTED_COLUMNS.items():
        if col not in df.columns:
            errors.append(f
        else:
            # Validación de tipo (suave)
            if dtype.startswith('datetime'):
                if not np.issubdtype(df[col].dtype, np.datetime64):
                    errors.append(f"Tipo inválido en {col}: {df[col].dtype}")
            elif dtype == 'object':
                pass
            else:
                if str(df[col].dtype) != dtype:
                    errors.append(f"Tipo inválido en {col}: {df[col].dtype} (esperado {dtype})")
    return (len(errors) == 0, errors)

def transform(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['anio'] = out['fecha_venta'].dt.year
    out['mes'] = out['fecha_venta'].dt.to_period('M').astype(str)
    out['ticket_unitario'] = out['total'] / out['cantidad']
    return out

print('✅ Validadores definidos')

### ✓ Validación de Esquema: Data Contract

**Concepto:** Validar que datos entrantes cumplan esquema esperado antes de procesar.

**Data Contract:**
- Diccionario con columnas esperadas y tipos de datos
- Valida presencia de columnas obligatorias
- Verifica tipos de datos (int64, float64, datetime64, object)

**Flujo:**
1. Leer CSV con `pd.read_csv()`
2. Normalizar fechas con `pd.to_datetime(errors='coerce')`
3. Validar esquema contra contrato
4. Si falla → logear errores y rechazar archivo
5. Si pasa → aplicar transformaciones

**Ventaja:** Detección temprana de problemas, evita corrupción downstream.

## 4. Carga y Métricas

In [ ]:
def load_to_sqlite(df: pd.DataFrame, table: str = 'ventas_near_rt') -> None:
    conn = sqlite3.connect(DB_PATH)
    try:
        df.to_sql(table, conn, if_exists='append', index=False)
    finally:
        conn.close()

def export_parquet(df: pd.DataFrame, base_name: str) -> str:
    ts = datetime.utcnow().strftime('%Y%m%dT%H%M%S')
    out_path = os.path.join(PARQUET_DIR, f
)
    df.to_parquet(out_path, index=False)
    return out_path

def compute_metrics(df: pd.DataFrame) -> Dict:
    return {
        'rows': len(df),
        'total_sum': float(df['total'].sum()),
        'ticket_promedio': float(df['total'].mean()),
        'clientes_unicos': int(df['cliente_id'].nunique())
    }

print('✅ Funciones de carga y métricas listas')

### 💾 Dual Output: SQLite + Parquet

**Concepto:** Persistir datos en múltiples formatos para diferentes casos de uso.

**SQLite:**
- `if_exists='append'`: añade registros sin sobrescribir
- Ideal para: queries SQL, análisis ad-hoc, dashboards pequeños
- Limitación: no escala para Big Data

**Parquet:**
- Formato columnar comprimido
- Timestamped: `ventas_20250101T120000.parquet`
- Ideal para: data lakes, analytics distribuidos (Spark, Athena)
- Ventaja: 10-100x más eficiente que CSV

**Estrategia:** SQLite para operacional, Parquet para histórico y analytics.

## 5. Worker con Reintentos y Backoff

In [ ]:
def process_file(path: str, max_retries: int = 3) -> bool:
    md5sum = file_md5(path)
    if is_processed(path, md5sum):
        logger.info(f'⏭️ Ya procesado: {os.path.basename(path)}')
        return True

    sleep = 1.0
    for attempt in range(1, max_retries+1):
        try:
            logger.info(f'📥 Procesando {os.path.basename(path)} (intento {attempt})')
            df = read_csv_safe(path)
            ok, errs = validate_schema(df)
            if not ok:
                raise ValueError('Esquema inválido: ' + '; '.join(errs))
            df_t = transform(df)
            load_to_sqlite(df_t)
            pq = export_parquet(df_t, base_name=os.path.splitext(os.path.basename(path))[0])
            m = compute_metrics(df_t)
            save_chkpt(path, md5sum, rows=len(df_t), status='ok')
            notify(f

### 🔄 Retry Logic con Exponential Backoff

**Concepto:** Reintentar operaciones fallidas con delays incrementales para manejar errores transitorios.

**Parámetros:**
- **max_retries:** número máximo de intentos (ej: 3)
- **base_delay:** delay inicial en segundos (ej: 2s)
- **backoff:** multiplicador entre intentos (ej: 2x)

**Ejemplo de delays:**
- Intento 1: 2s
- Intento 2: 4s (2 × 2)
- Intento 3: 8s (4 × 2)

**Cuándo usar:**
- Errores de red/conexión
- Rate limits de APIs
- Locks de base de datos
- Servicios externos temporalmente caídos

**Nota:** No reintentar errores lógicos (validación fallida, datos corruptos).